# 参数管理
选择模型架构并设置了超参数后，就进入了训练阶段，我们的目标是找到使损失函数最小化的模型参数.
- 访问参数
- 参数初始化
- 在不同模型组件间共享参数

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.3744],
        [0.3979]], grad_fn=<AddmmBackward0>)

In [5]:
print(net[1])
print(net[2])
print(net)
print(net[2].state_dict())
print(net[1].state_dict())

ReLU()
Linear(in_features=8, out_features=1, bias=True)
Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)
OrderedDict([('weight', tensor([[ 0.1012,  0.0116, -0.0551, -0.2913,  0.2065, -0.0875,  0.1161, -0.0879]])), ('bias', tensor([0.2342]))])
OrderedDict()


In [6]:
print(*[(name, param.shape) for name,param in net[0].named_parameters()])
print(*[(name, param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


在python当中*为列表，**代表字典

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)


tensor([[0.0984],
        [0.0984]], grad_fn=<AddmmBackward0>)

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 5.2.2 参数初始化
通过`net.apply(init_normal_func())`来进行网络参数的初始化，其中`init_normal_funct()`为自定义的参数初始化函数

In [10]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean = 0, std = 0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0158, -0.0068, -0.0022, -0.0043]), tensor(0.))